In [1]:
import os
import re
import requests
from PIL import Image
import pandas as pd
import numpy as np

# Libraries for data visualization
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Deep Learnong Libraries, Preprocsseing, Modeling, Evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D,MaxPooling2D, Dropout, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing import image
from tensorflow import keras
from tensorflow.keras.optimizers.legacy import Adam, Adamax
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16

from tensorflow.keras import regularizers
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_data_dir = '/Users/aanshsavla/Desktop/Aansh/AI/ML/DataSets/chest_xray/chest_xray/train'
filepaths = []
labels = []

folds = os.listdir(train_data_dir)
for fold in folds:
    if str(fold) != '.DS_Store':
        foldpath = os.path.join(train_data_dir,fold)
        filelist = os.listdir(foldpath)
        for file in filelist:
            fpath = os.path.join(foldpath,file)
            filepaths.append(fpath)
            labels.append(fold)
# Concatenate data paths with labels into one dataframe
Fseries = pd.Series(filepaths,name='filepaths')
Lseries = pd.Series(labels, name='labels')

train_df = pd.concat([Fseries,Lseries],axis=1)

In [3]:
val_data_dir = '/Users/aanshsavla/Desktop/Aansh/AI/ML/DataSets/chest_xray/chest_xray/val'
filepaths= []
labels = []

folds = os.listdir(val_data_dir)
for fold in folds:
    if str(fold) != '.DS_Store':
        foldpath = os.path.join(val_data_dir,fold)
        filelist = os.listdir(foldpath)
        for file in filelist:
            fpath = os.path.join(foldpath,file)
            filepaths.append(fpath)
            labels.append(fold)

# Concatenate data paths with labels into one dataframe
Fseries = pd.Series(filepaths, name='filepaths')
Lseries = pd.Series(labels, name='labels')
val_df = pd.concat([Fseries,Lseries],axis=1)

In [4]:
img_width = 244
img_height = 244

In [5]:
conv_base = VGG16(weights='imagenet', 
                  include_top=False,
                  input_shape=(img_width, img_height, 3))

# Show architecture
conv_base.summary()

2024-02-19 17:59:27.804428: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2024-02-19 17:59:27.804451: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-02-19 17:59:27.804457: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-02-19 17:59:27.804495: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-19 17:59:27.804513: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 244, 244, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 244, 244, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 244, 244, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 122, 122, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 122, 122, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 122, 122, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 61, 61, 128)       0     

In [6]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 16

def extract_features(df, sample_count):
    features = np.zeros(shape=(sample_count, 7, 7, 512))  # Must be equal to the output of the convolutional base
    labels = np.zeros(shape=(sample_count,2))
    # Preprocess data
    generator = datagen.flow_from_dataframe(df,
                                            target_size=(img_width,img_height),
                                            x_col='filepaths',y_col='labels',
                                            batch_size = batch_size,
                                            class_mode='categorical',
                                            color_mode = 'rgb')
    # Pass data through convolutional base
    i = 0
    for inputs_batch, labels_batch in generator:
        with tf.device("/cpu:0"):
            features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size: (i + 1) * batch_size] = features_batch
        labels[i * batch_size: (i + 1) * batch_size] = labels_batch
        i += 1
        print(i * batch_size)
        if i * batch_size >= sample_count:
            break
    return features, labels

In [7]:
train_features, train_labels = extract_features(train_df, 5216)  # Agree with our small dataset size
validation_features, validation_labels = extract_features(val_df, 16)

Found 5216 validated image filenames belonging to 2 classes.


2024-02-19 17:59:33.763319: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 1s/step
16
1/1 [==============================] - 1s 1s/step
32
1/1 [==============================] - 1s 1s/step
48
1/1 [==============================] - 1s 1s/step
64
1/1 [==============================] - 1s 1s/step
80
1/1 [==============================] - 1s 1s/step
96
1/1 [==============================] - 1s 1s/step
112
1/1 [==============================] - 1s 1s/step
128
1/1 [==============================] - 1s 1s/step
144
1/1 [==============================] - 1s 1s/step
160
1/1 [==============================] - 1s 1s/step
176
1/1 [==============================] - 1s 1s/step
192
1/1 [==============================] - 1s 1s/step
208
1/1 [==============================] - 1s 1s/step
224
1/1 [==============================] - 1s 1s/step
240
1/1 [==============================] - 1s 1s/step
256
1/1 [==============================] - 1s 1s/step
272
1/1 [==============================] - 1s 1s/step
288
1/1 [=============================

1/1 [==============================] - 1s 1s/step
2416
1/1 [==============================] - 1s 1s/step
2432
1/1 [==============================] - 1s 1s/step
2448
1/1 [==============================] - 1s 1s/step
2464
1/1 [==============================] - 1s 1s/step
2480
1/1 [==============================] - 1s 1s/step
2496
1/1 [==============================] - 1s 1s/step
2512
1/1 [==============================] - 1s 1s/step
2528
1/1 [==============================] - 1s 1s/step
2544
1/1 [==============================] - 1s 1s/step
2560
1/1 [==============================] - 1s 1s/step
2576
1/1 [==============================] - 1s 1s/step
2592
1/1 [==============================] - 1s 1s/step
2608
1/1 [==============================] - 1s 1s/step
2624
1/1 [==============================] - 1s 1s/step
2640
1/1 [==============================] - 1s 1s/step
2656
1/1 [==============================] - 1s 1s/step
2672
1/1 [==============================] - 1s 1s/step
2688
1/1 [=====

1/1 [==============================] - 1s 1s/step
4800
1/1 [==============================] - 1s 1s/step
4816
1/1 [==============================] - 1s 1s/step
4832
1/1 [==============================] - 1s 1s/step
4848
1/1 [==============================] - 1s 1s/step
4864
1/1 [==============================] - 1s 1s/step
4880
1/1 [==============================] - 1s 1s/step
4896
1/1 [==============================] - 1s 1s/step
4912
1/1 [==============================] - 1s 1s/step
4928
1/1 [==============================] - 1s 1s/step
4944
1/1 [==============================] - 1s 1s/step
4960
1/1 [==============================] - 1s 1s/step
4976
1/1 [==============================] - 1s 1s/step
4992
1/1 [==============================] - 1s 1s/step
5008
1/1 [==============================] - 1s 1s/step
5024
1/1 [==============================] - 1s 1s/step
5040
1/1 [==============================] - 1s 1s/step
5056
1/1 [==============================] - 1s 1s/step
5072
1/1 [=====

In [8]:
model = Sequential()
model.add(GlobalAveragePooling2D(input_shape=(7,7,512)))
model.add(Dense(2, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 global_average_pooling2d (  (None, 512)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 2)                 1026      
                                                                 
Total params: 1026 (4.01 KB)
Trainable params: 1026 (4.01 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
epochs = 10

In [10]:
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])

# Train model
with tf.device("/cpu:0"):
    history = model.fit(train_features, train_labels,
                    epochs=epochs,
                    batch_size=batch_size, 
                    validation_data=(validation_features, validation_labels))

Epoch 1/10
326/326 [==============================] - 1s 3ms/step - loss: 0.3526 - acc: 0.8407 - val_loss: 0.6126 - val_acc: 0.6250
Epoch 2/10
326/326 [==============================] - 1s 2ms/step - loss: 0.2381 - acc: 0.9133 - val_loss: 0.7597 - val_acc: 0.6250
Epoch 3/10
326/326 [==============================] - 1s 3ms/step - loss: 0.2048 - acc: 0.9287 - val_loss: 0.6091 - val_acc: 0.6875
Epoch 4/10
326/326 [==============================] - 1s 2ms/step - loss: 0.1864 - acc: 0.9346 - val_loss: 0.7263 - val_acc: 0.6250
Epoch 5/10
326/326 [==============================] - 1s 2ms/step - loss: 0.1729 - acc: 0.9388 - val_loss: 0.7364 - val_acc: 0.6250
Epoch 6/10
326/326 [==============================] - 1s 3ms/step - loss: 0.1618 - acc: 0.9421 - val_loss: 0.7512 - val_acc: 0.6250
Epoch 7/10
326/326 [==============================] - 1s 2ms/step - loss: 0.1535 - acc: 0.9465 - val_loss: 0.7855 - val_acc: 0.6250
Epoch 8/10
326/326 [==============================] - 1s 2ms/step - loss: 0.